In [1]:
from dask.distributed import Client, LocalCluster
from dask_jobqueue import PBSCluster
from distributed.utils import tmpfile
import os

/jobfs/91154998.gadi-pbs/ipykernel_1826046/2795989920.py:3: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
os.environ['DASK_PYTHON'] = '/scratch/vp91/Training/environment/bin/python'
setup_commands = ["module load python3/3.9.2", "source /scratch/vp91/Training/environment/bin/activate"]
extra = ['-q normal', '-P vp91', '-l ncpus=48', '-l mem=192GB']

In [4]:
cluster = PBSCluster(walltime="00:50:00", 
                     cores=48, 
                     memory="192GB",
                     #processes=48, 
                     shebang='#!/usr/bin/env bash',
                     job_extra=extra, 
                     local_directory='$TMPDIR', 
                     header_skip=["select"], 
                     interface="ib0",
                     env_extra=setup_commands,
                     python=os.environ["DASK_PYTHON"])

/home/659/jj8451/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/659/jj8451/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:274: FutureWarning: env_extra has been renamed to job_script_prologue. You are still using it (even if only set to []; please also check config files). If you did not set job_script_prologue yet, env_extra will be respected for now, but it will be removed in a future release. If you already set job_script_prologue, env_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/659/jj8451/.local/lib/python3.11/site-package

In [5]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -l walltime=00:50:00
#PBS -q normal
#PBS -P vp91
#PBS -l ncpus=48
#PBS -l mem=192GB
module load python3/3.9.2
source /scratch/vp91/Training/environment/bin/activate
/scratch/vp91/Training/environment/bin/python -m distributed.cli.dask_worker tcp://10.6.42.72:43895 --nthreads 6 --nworkers 8 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0



/home/659/jj8451/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/659/jj8451/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:274: FutureWarning: env_extra has been renamed to job_script_prologue. You are still using it (even if only set to []; please also check config files). If you did not set job_script_prologue yet, env_extra will be respected for now, but it will be removed in a future release. If you already set job_script_prologue, env_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/659/jj8451/.local/lib/python3.11/site-package

In [6]:
cluster.scale(2)

In [7]:
cluster

Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.42.72:43895,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [8]:
client = Client(cluster)

In [11]:
client

<Client: 'tcp://10.6.42.72:43895' processes=8 threads=48, memory=178.80 GiB>

In [10]:
!qstat

Job id                 Name             User              Time Use S Queue
---------------------  ---------------- ----------------  -------- - -----
91144733.gadi-pbs      Irr_GEMM_2023-0* jj8451            02:05:54 R gpuvolta-exec   
91147399.gadi-pbs      sys-dashboard-s* jj8451            00:00:09 R normal-exec     
91154434.gadi-pbs      dask-worker      jj8451            00:02:17 R normal-exec     
91154998.gadi-pbs      sys-dashboard-s* jj8451            00:00:12 R normal-exec     
91155646.gadi-pbs      dask-worker      jj8451            00:00:00 R normal-exec     


In [12]:
!pwd

/home/659/jj8451
